In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir -p /content/drive/MyDrive/gpt2_spoc_model

In [3]:
!pip install pandas tqdm wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=9379dea0910154ca287e7b26446687cd2d435f663cde55aebe561d6060a35c11
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [4]:
!mkdir -p data
!wget https://sumith1896.github.io/spoc/data/spoc.zip -O spoc.zip
!unzip -o spoc.zip -d data

!mv data/spoc data/spoc_dataset
!rm spoc.zip

--2025-10-31 17:35:50--  https://sumith1896.github.io/spoc/data/spoc.zip
Resolving sumith1896.github.io (sumith1896.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to sumith1896.github.io (sumith1896.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9960355 (9.5M) [application/x-zip-compressed]
Saving to: ‘spoc.zip’

spoc.zip            100%[===================>]   9.50M  --.-KB/s    in 0.06s   

2025-10-31 17:35:51 (147 MB/s) - ‘spoc.zip’ saved [9960355/9960355]

Archive:  spoc.zip
   creating: data/test/
  inflating: data/test/spoc-testw.tsv  
  inflating: data/test/spoc-testp.tsv  
   creating: data/testcases/
   creating: data/testcases/1000A/
  inflating: data/testcases/1000A/1000A_testcases.txt  
  inflating: data/testcases/1000A/1000A_testcases_hidden.txt  
  inflating: data/testcases/1000A/1000A_testcases_public.txt  
   creating: data/testcases/1003A/
  inflating: data/testcases/1003A/1003A_test

In [5]:
!ls data


LICENSE  README.md  test  testcases  train


In [6]:
!mkdir -p data/spoc_dataset
!mv data/train data/test data/testcases data/spoc_dataset/
!ls data/spoc_dataset/train/split


spoc-train-eval.tsv  spoc-train-test.tsv  spoc-train-train.tsv


In [7]:
import pandas as pd

train_path = "data/spoc_dataset/train/split/spoc-train-train.tsv"
eval_path = "data/spoc_dataset/train/split/spoc-train-eval.tsv"
test_path = "data/spoc_dataset/train/split/spoc-train-test.tsv"

df_train = pd.read_csv(train_path, sep='\t', header=None, names=['pseudo','code'])
df_eval = pd.read_csv(eval_path, sep='\t', header=None, names=['pseudo','code'])
df_test = pd.read_csv(test_path, sep='\t', header=None, names=['pseudo','code'])

print("✅ Train size:", len(df_train))
print("✅ Eval size:", len(df_eval))
print("✅ Test size:", len(df_test))
df_train.head()


/tmp/ipython-input-1798567749.py:7: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(train_path, sep='\t', header=None, names=['pseudo','code'])


✅ Train size: 246087
✅ Eval size: 27289
✅ Test size: 20481


,,,,,pseudo,code
text,code,workerid,probid,subid,line,indent
NaN,int main() {,01,3A,41470897,0,0
create string s,string s;,01,3A,41470897,1,1
"create integers x1, y1, x2, y2","int x1, y1, x2, y2;",01,3A,41470897,2,1
read s,cin >> s;,01,3A,41470897,3,1


In [8]:
import pandas as pd

train_path = "data/spoc_dataset/train/split/spoc-train-train.tsv"

# Load with low_memory=False to suppress the warning
df_train_raw = pd.read_csv(train_path, sep='\t', low_memory=False)
print(df_train_raw.head(10))
print("\nColumns:", df_train_raw.columns.tolist())


                                                text  \
0                                                NaN   
1                                    create string s   
2                     create integers x1, y1, x2, y2   
3                                             read s   
4                                set x1 to s[0] - 96   
5                               set y1 to s[1] - '0'   
6                                             read s   
7                                set x2 to s[0] - 96   
8                               set y2 to s[1] - '0'   
9  print maximum of absolute value of x1 - x2 and...   

                                               code  workerid probid  \
0                                      int main() {         1     3A   
1                                         string s;         1     3A   
2                               int x1, y1, x2, y2;         1     3A   
3                                         cin >> s;         1     3A   
4                      

In [9]:
df_train = df_train_raw[['text', 'code']].dropna().reset_index(drop=True)
df_train.columns = ['pseudo', 'code']

print("✅ Cleaned Train size:", len(df_train))
df_train.head()


✅ Cleaned Train size: 181862


,pseudo,code
0,create string s,string s;
1,"create integers x1, y1, x2, y2","int x1, y1, x2, y2;"
2,read s,cin >> s;
3,set x1 to s[0] - 96,x1 = s[0] - 96;
4,set y1 to s[1] - '0',y1 = s[1] - '0';


In [10]:
df_train['formatted'] = (
    "### PSEUDOCODE:\n" + df_train['pseudo'].astype(str) +
    "\n### PYTHON CODE:\n" + df_train['code'].astype(str)
)

df_train['formatted'].to_csv("spoc_train_formatted.txt", index=False, header=False)

print("✅ Saved formatted dataset: spoc_train_formatted.txt")


✅ Saved formatted dataset: spoc_train_formatted.txt


# Fine-Tune GPT-2

In [12]:
"""
COLAB-OPTIMIZED GPT-2 FINE-TUNING FOR PSEUDO-CODE TO CODE
=========================================================
This notebook is optimized for Google Colab with limited GPU/CPU resources.
- Uses smaller dataset (5000 train, 1000 eval)
- Reduced batch sizes and epochs
- Memory-efficient training
"""

# ========================
# SECTION 1: SETUP & INSTALLATION
# ========================

print("📦 Installing required packages...")
!pip install transformers datasets accelerate torch evaluate -q

import os
import pandas as pd
import torch
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, DatasetDict
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n✅ Using device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ========================
# SECTION 2: CONFIGURATION
# ========================

class Config:
    """Configuration for 12 GB GPU fine-tuning"""
    TRAIN_PATH = "data/spoc_dataset/train/split/spoc-train-train.tsv"
    OUTPUT_DIR = "/content/drive/MyDrive/gpt2_spoc_colab"


    # Use more data
    TRAIN_SIZE = 8000       # (from 5k → 20k)
    EVAL_SIZE = 2000

    # Larger context window — GPT-2 can handle up to 1024 tokens
    MAX_LENGTH = 256         # (from 128 → 256)

    # Model — still GPT-2 small (124M params) but you can go higher
    MODEL_NAME = "gpt2-medium"  # 355M params (optional)

    # Training
    BATCH_SIZE = 2           # smaller batch to fit the larger model
    GRADIENT_ACCUMULATION = 8  # effective batch size = 16
    LEARNING_RATE = 5e-5
    NUM_EPOCHS = 1           # train longer for deeper convergence
    WARMUP_STEPS = 200

    # Evaluation
    EVAL_STEPS = 500
    SAVE_STEPS = 500
    LOGGING_STEPS = 100

config = Config()

# ========================
# SECTION 3: DATA PREPARATION
# ========================

def load_dataset_colab(train_path, config):
    """Load and prepare dataset for Colab training"""
    print("\n📊 Loading dataset...")

    # Load data
    df = pd.read_csv(train_path, sep='\t', low_memory=False)
    df = df[['text', 'code']].dropna().reset_index(drop=True)
    df.columns = ['pseudo', 'code']

    print(f"   Total samples: {len(df)}")

    # Format data
    df['formatted'] = (
        "### PSEUDOCODE:\n" + df['pseudo'].astype(str) +
        "\n### PYTHON CODE:\n" + df['code'].astype(str)
    )

    # Create train/eval split
    total_size = config.TRAIN_SIZE + config.EVAL_SIZE
    df_subset = df.head(total_size)

    train_data = df_subset['formatted'][:config.TRAIN_SIZE].tolist()
    eval_data = df_subset['formatted'][config.TRAIN_SIZE:].tolist()

    # Create dataset
    dataset = DatasetDict({
        'train': Dataset.from_dict({'text': train_data}),
        'eval': Dataset.from_dict({'text': eval_data})
    })

    print(f"✅ Train size: {len(dataset['train'])}")
    print(f"✅ Eval size: {len(dataset['eval'])}")

    return dataset

# Load dataset
dataset = load_dataset_colab(config.TRAIN_PATH, config)

# Show sample
print("\n📝 Sample data:")
print(dataset['train'][0]['text'][:200] + "...")

# ========================
# SECTION 4: MODEL & TOKENIZER
# ========================

print("\n🤖 Loading GPT-2 model and tokenizer...")

tokenizer = GPT2Tokenizer.from_pretrained(config.MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(config.MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)

print(f"✅ Model loaded: {model.num_parameters():,} parameters")

# ========================
# SECTION 5: TOKENIZATION
# ========================

print("\n🔤 Tokenizing dataset...")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=config.MAX_LENGTH,
        return_tensors=None
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing"
)

print("✅ Tokenization complete")

# ========================
# SECTION 6: TRAINING SETUP
# ========================

print("\n⚙️ Setting up training...")

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training arguments (Colab-optimized)
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,

    # Training
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION,
    learning_rate=config.LEARNING_RATE,
    warmup_steps=config.WARMUP_STEPS,
    weight_decay=0.01,

    # Evaluation
    eval_strategy="steps",
    eval_steps=config.EVAL_STEPS,
    per_device_eval_batch_size=config.BATCH_SIZE,

    # Saving
    save_strategy="steps",
    save_steps=config.SAVE_STEPS,
    save_total_limit=2,
    load_best_model_at_end=True,

    # Logging
    logging_steps=config.LOGGING_STEPS,
    logging_dir=f"{config.OUTPUT_DIR}/logs",

    # Optimization
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,

    # Misc
    report_to="none",
    seed=42
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

print("✅ Training setup complete")

# ========================
# SECTION 7: TRAINING
# ========================

print("\n🚀 Starting training...")
print("=" * 60)

# Train
train_result = trainer.train()

# Print results
print("\n" + "=" * 60)
print("📊 TRAINING RESULTS")
print("=" * 60)
print(f"Final Loss: {train_result.metrics.get('train_loss', 'N/A'):.4f}")
print(f"Training Time: {train_result.metrics.get('train_runtime', 0):.2f}s")
print("=" * 60)

# ========================
# SECTION 8: SAVE MODEL
# ========================

print("\n💾 Saving model...")

final_path = f"{config.OUTPUT_DIR}/final"
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)


print(f"✅ Model saved to: {final_path}")

# ========================
# SECTION 9: TESTING
# ========================

print("\n🧪 Testing model generation...")
print("=" * 60)

model.eval()

test_cases = [
    "create integer variable x",
    "read input from user",
    "for i from 0 to 10 print i",
    "if x greater than 5 print yes",
    "create list numbers"
]

def generate_code(pseudo_code):
    prompt = f"### PSEUDOCODE:\n{pseudo_code}\n### PYTHON CODE:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### PYTHON CODE:" in generated:
        code = generated.split("### PYTHON CODE:")[1].strip()
        if "###" in code:
            code = code.split("###")[0].strip()
        return code

    return generated

for i, test in enumerate(test_cases, 1):
    print(f"\n--- Test {i} ---")
    print(f"Pseudo: {test}")
    code = generate_code(test)
    print(f"Code:   {code}")
    print("-" * 40)

# ========================
# SECTION 10: EVALUATION METRICS
# ========================

print("\n📈 Computing evaluation metrics...")

# Evaluate on eval set
eval_results = trainer.evaluate()

print("\n" + "=" * 60)
print("📊 EVALUATION METRICS")
print("=" * 60)
print(f"Eval Loss: {eval_results['eval_loss']:.4f}")
print(f"Perplexity: {torch.exp(torch.tensor(eval_results['eval_loss'])):.2f}")
print("=" * 60)

# ========================
# SECTION 11: DOWNLOAD MODEL (Optional)
# ========================

print("\n📥 To download the model, run:")
print(f"   from google.colab import files")
print(f"   !zip -r gpt2-spoc-model.zip {final_path}")
print(f"   files.download('gpt2-spoc-model.zip')")

# ========================
# QUICK START FOR INFERENCE
# ========================

print("\n" + "=" * 60)
print("✨ QUICK START FOR INFERENCE")
print("=" * 60)
print("""
# Load the trained model:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-spoc-colab/final')
model = GPT2LMHeadModel.from_pretrained('./gpt2-spoc-colab/final')

# Generate code:
pseudo = "create integer variable x with value 10"
prompt = f"### PSEUDOCODE:\\n{pseudo}\\n### PYTHON CODE:\\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(code)
""")

print("\n✅ All done! Your model is ready to use! 🎉")

📦 Installing required packages...

✅ Using device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB

📊 Loading dataset...
   Total samples: 181862
✅ Train size: 8000
✅ Eval size: 2000

📝 Sample data:
### PSEUDOCODE:
create string s
### PYTHON CODE:
string s;...

🤖 Loading GPT-2 model and tokenizer...
✅ Model loaded: 354,823,168 parameters

🔤 Tokenizing dataset...


Tokenizing:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing:   0%|          | 0/2000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


✅ Tokenization complete

⚙️ Setting up training...
✅ Training setup complete

🚀 Starting training...


Step,Training Loss,Validation Loss
500,0.483000,0.711494


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].



📊 TRAINING RESULTS
Final Loss: 0.7491
Training Time: 782.93s

💾 Saving model...
✅ Model saved to: /content/drive/MyDrive/gpt2_spoc_colab/final

🧪 Testing model generation...

--- Test 1 ---
Pseudo: create integer variable x
Code:   int x;
----------------------------------------

--- Test 2 ---
Pseudo: read input from user
Code:   cin >> input >> endl;
----------------------------------------

--- Test 3 ---
Pseudo: for i from 0 to 10 print i
Code:   for (int i = 0; i < 10; i++) { cout << i; }
----------------------------------------

--- Test 4 ---
Pseudo: if x greater than 5 print yes
Code:   if (x > 5) cout << yes;
----------------------------------------

--- Test 5 ---
Pseudo: create list numbers
Code:   vector<int> numbers;
----------------------------------------

📈 Computing evaluation metrics...



📊 EVALUATION METRICS
Eval Loss: 0.7115
Perplexity: 2.04

📥 To download the model, run:
   from google.colab import files
   !zip -r gpt2-spoc-model.zip /content/drive/MyDrive/gpt2_spoc_colab/final
   files.download('gpt2-spoc-model.zip')

✨ QUICK START FOR INFERENCE

# Load the trained model:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-spoc-colab/final')
model = GPT2LMHeadModel.from_pretrained('./gpt2-spoc-colab/final')

# Generate code:
pseudo = "create integer variable x with value 10"
prompt = f"### PSEUDOCODE:\n{pseudo}\n### PYTHON CODE:\n"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(code)


✅ All done! Your model is ready to use! 🎉


In [13]:
# Save compressed copy to Drive
!zip -r /content/drive/MyDrive/gpt2_spoc_model.zip {final_path}
print("✅ Model zipped and saved to Drive as gpt2_spoc_model.zip")


  adding: content/drive/MyDrive/gpt2_spoc_colab/final/ (stored 0%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/config.json (deflated 53%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/generation_config.json (deflated 31%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/tokenizer_config.json (deflated 56%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/special_tokens_map.json (deflated 74%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/vocab.json (deflated 68%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/merges.txt (deflated 53%)
  adding: content/drive/MyDrive/gpt2_spoc_colab/final/training_args.bin (deflated 54%)
✅ Model zipped and saved to Drive as gpt2_spoc_model.zip


# Evalutaion

In [20]:
"""
COLAB-OPTIMIZED EVALUATION SCRIPT
===================================
Lightweight evaluation for limited resources
"""

# ========================
# SETUP
# ========================

!pip install sacrebleu evaluate -q

import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
import evaluate
import json

# ========================
# LOAD MODEL
# ========================

print("🔄 Loading trained model...")

MODEL_PATH = "/content/drive/MyDrive/gpt2_spoc_colab/final"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_PATH)
model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)
model.to(device)
model.eval()

print(f"✅ Model loaded on {device}")

# ========================
# LOAD TEST DATA
# ========================

print("\n📊 Loading test data...")

TEST_PATH = "data/spoc_dataset/train/split/spoc-train-test.tsv"
NUM_SAMPLES = 200  # Reduced for Colab

df_test = pd.read_csv(TEST_PATH, sep='\t', low_memory=False)
df_test = df_test[['text', 'code']].dropna().reset_index(drop=True)
df_test.columns = ['pseudo', 'code']
df_test = df_test.head(NUM_SAMPLES)

print(f"✅ Loaded {len(df_test)} test samples")

# ========================
# GENERATE PREDICTIONS
# ========================

print("\n🤖 Generating predictions...")

predictions = []
references = df_test['code'].tolist()

for pseudo in tqdm(df_test['pseudo'], desc="Generating"):
    prompt = f"### PSEUDOCODE:\n{pseudo}\n### PYTHON CODE:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### PYTHON CODE:" in generated:
        code = generated.split("### PYTHON CODE:")[1].strip()
        if "###" in code:
            code = code.split("###")[0].strip()
        predictions.append(code)
    else:
        predictions.append(generated)

print(f"✅ Generated {len(predictions)} predictions")

# ========================
# COMPUTE METRICS
# ========================

print("\n📈 Computing metrics...")

# 1. BLEU Score
bleu = evaluate.load("sacrebleu")
formatted_refs = [[ref] for ref in references]
bleu_results = bleu.compute(predictions=predictions, references=formatted_refs)

# 2. Exact Match
exact_matches = sum(p.strip() == r.strip() for p, r in zip(predictions, references))
exact_match_acc = exact_matches / len(predictions) * 100

# 3. Syntax Accuracy
valid_syntax = 0
for code in predictions:
    try:
        compile(code, '<string>', 'exec')
        valid_syntax += 1
    except:
        pass
syntax_acc = valid_syntax / len(predictions) * 100

# 4. Token Accuracy
correct_tokens = 0
total_tokens = 0

for pred, ref in zip(predictions, references):
    pred_tokens = tokenizer.encode(pred)
    ref_tokens = tokenizer.encode(ref)

    min_len = min(len(pred_tokens), len(ref_tokens))
    correct_tokens += sum(p == r for p, r in zip(pred_tokens[:min_len], ref_tokens[:min_len]))
    total_tokens += max(len(pred_tokens), len(ref_tokens))

token_acc = correct_tokens / total_tokens * 100 if total_tokens > 0 else 0

# ========================
# DISPLAY RESULTS
# ========================

print("\n" + "=" * 60)
print("📊 EVALUATION RESULTS")
print("=" * 60)
print(f"Samples Evaluated: {len(predictions)}")
print(f"\nBLEU Score: {bleu_results['score']:.2f}")
print(f"Exact Match: {exact_match_acc:.2f}%")
print(f"Syntax Accuracy: {syntax_acc:.2f}%")
print(f"Token Accuracy: {token_acc:.2f}%")
print("=" * 60)

# ========================
# SAVE RESULTS
# ========================

print("\n💾 Saving results...")

# Save metrics
metrics = {
    "bleu": bleu_results['score'],
    "exact_match": exact_match_acc,
    "syntax_accuracy": syntax_acc,
    "token_accuracy": token_acc,
    "num_samples": len(predictions)
}

with open("colab_evaluation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save predictions
results_df = pd.DataFrame({
    'pseudo_code': df_test['pseudo'],
    'reference': references,
    'prediction': predictions
})
results_df.to_csv("colab_evaluation_results.csv", index=False)

print("✅ Results saved:")
print("   - colab_evaluation_metrics.json")
print("   - colab_evaluation_results.csv")

# ========================
# SHOW SAMPLE PREDICTIONS
# ========================

print("\n" + "=" * 60)
print("📝 SAMPLE PREDICTIONS")
print("=" * 60)

# Show 5 random samples
samples = results_df.sample(n=min(5, len(results_df)))

for idx, row in samples.iterrows():
    is_match = row['reference'].strip() == row['prediction'].strip()

    print(f"\n--- Sample {idx} ---")
    print(f"Pseudo: {row['pseudo_code']}")
    print(f"Ref:    {row['reference']}")
    print(f"Pred:   {row['prediction']}")
    print(f"Match:  {'✅' if is_match else '❌'}")
    print("-" * 40)

# ========================
# ANALYSIS
# ========================

print("\n" + "=" * 60)
print("🔍 ANALYSIS")
print("=" * 60)

# Calculate average lengths
avg_ref_len = sum(len(r) for r in references) / len(references)
avg_pred_len = sum(len(p) for p in predictions) / len(predictions)

print(f"Average Reference Length: {avg_ref_len:.1f} chars")
print(f"Average Prediction Length: {avg_pred_len:.1f} chars")
print(f"Length Ratio: {avg_pred_len/avg_ref_len:.2f}")

# Count perfect matches
perfect_matches = sum(1 for p, r in zip(predictions, references) if p.strip() == r.strip())
print(f"\nPerfect Matches: {perfect_matches} ({perfect_matches/len(predictions)*100:.1f}%)")

# Count syntax errors
syntax_errors = len(predictions) - valid_syntax
print(f"Syntax Errors: {syntax_errors} ({syntax_errors/len(predictions)*100:.1f}%)")

print("=" * 60)

# ========================
# DOWNLOAD RESULTS
# ========================

print("\n📥 To download results, run:")
print("""
from google.colab import files
files.download('colab_evaluation_metrics.json')
files.download('colab_evaluation_results.csv')
""")

print("\n✅ Evaluation complete! 🎉")

🔄 Loading trained model...
✅ Model loaded on cuda

📊 Loading test data...
✅ Loaded 200 test samples

🤖 Generating predictions...


Generating: 100%|██████████| 200/200 [08:04<00:00,  2.42s/it]


✅ Generated 200 predictions

📈 Computing metrics...



📊 EVALUATION RESULTS
Samples Evaluated: 200

BLEU Score: 57.22
Exact Match: 22.00%
Syntax Accuracy: 7.00%
Token Accuracy: 47.60%

💾 Saving results...
✅ Results saved:
   - colab_evaluation_metrics.json
   - colab_evaluation_results.csv

📝 SAMPLE PREDICTIONS

--- Sample 95 ---
Pseudo: read n
Ref:    cin >> n;
Pred:   cin >> n; }
Match:  ❌
----------------------------------------

--- Sample 15 ---
Pseudo: print integer casted size of ans print newline
Ref:    cout << (int)ans.size() << endl;
Pred:   cout << int(ans.size()) << endl; }
Match:  ❌
----------------------------------------

--- Sample 30 ---
Pseudo: print count + 1
Ref:    cout << count + 1 << endl;
Pred:   cout << count + 1; } } } }
Match:  ❌
----------------------------------------

--- Sample 158 ---
Pseudo: else
Ref:    else
Pred:   else { }
Match:  ❌
----------------------------------------

--- Sample 128 ---
Pseudo: create integer n
Ref:    int n;
Pred:   int n;
Match:  ✅
----------------------------------------

🔍 AN